In [ ]:
import sqlite3

def create_connection(db_file):
    conn = sqlite3.connect(db_file)
    return conn

def create_tables(conn):
    create_customers_table = """
    CREATE TABLE IF NOT EXISTS customers (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        email TEXT NOT NULL UNIQUE,
        phone TEXT NOT NULL
    );"""

    create_purchases_table = """
    CREATE TABLE IF NOT EXISTS purchases (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        customer_id INTEGER,
        item TEXT NOT NULL,
        amount REAL NOT NULL,
        purchase_date TEXT NOT NULL,
        FOREIGN KEY (customer_id) REFERENCES customers (id)
    );"""

    create_interactions_table = """
    CREATE TABLE IF NOT EXISTS interactions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        customer_id INTEGER,
        interaction_type TEXT NOT NULL,
        interaction_date TEXT NOT NULL,
        notes TEXT,
        FOREIGN KEY (customer_id) REFERENCES customers (id)
    );"""

    conn.execute(create_customers_table)
    conn.execute(create_purchases_table)
    conn.execute(create_interactions_table)

conn = create_connection("customer_data.db")
create_tables(conn)

def insert_customer(conn, name, email, phone):
    sql = '''INSERT INTO customers(name, email, phone) VALUES(?,?,?)'''
    cur = conn.cursor()
    cur.execute(sql, (name, email, phone))
    conn.commit()
    return cur.lastrowid

def insert_purchase(conn, customer_id, item, amount, purchase_date):
    sql = '''INSERT INTO purchases(customer_id, item, amount, purchase_date) VALUES(?,?,?,?)'''
    cur = conn.cursor()
    cur.execute(sql, (customer_id, item, amount, purchase_date))
    conn.commit()

def insert_interaction(conn, customer_id, interaction_type, interaction_date, notes):
    sql = '''INSERT INTO interactions(customer_id, interaction_type, interaction_date, notes) VALUES(?,?,?,?)'''
    cur = conn.cursor()
    cur.execute(sql, (customer_id, interaction_type, interaction_date, notes))
    conn.commit()

conn = create_connection("customer_data.db")


"\ncustomer_id = insert_customer(conn, 'John Doe', 'john.doe@example.com', '555-5555')\ninsert_purchase(conn, customer_id, 'Product A', 99.99, '2024-07-15')\ninsert_interaction(conn, customer_id, 'Email', '2024-07-15', 'Inquired about Product A')\n"

In [ ]:
!pip install faker

In [ ]:
import faker
from faker import Faker
import random

fake = Faker()

# Add 10 more customers
for _ in range(100):
    name = fake.name()
    email = fake.email()
    phone = fake.phone_number()
    customer_id = insert_customer(conn, name, email, phone)

    # Add 1-3 purchases for each customer
    for _ in range(random.randint(1, 3)):
        item = fake.word()  # Generate random item names
        amount = round(random.uniform(10, 200), 2)
        purchase_date = fake.date_between(start_date='-1y', end_date='today')
        insert_purchase(conn, customer_id, item, amount, purchase_date.strftime('%Y-%m-%d'))

    # Add 1-2 interactions for each customer
    for _ in range(random.randint(1, 2)):
        interaction_type = random.choice(['Email', 'Phone', 'Chat'])
        interaction_date = fake.date_between(start_date='-1y', end_date='today')
        notes = fake.sentence()
        insert_interaction(conn, customer_id, interaction_type, interaction_date.strftime('%Y-%m-%d'), notes)

In [ ]:
# Fetch data from the customers table
customers_df = pd.read_sql_query("SELECT * FROM customers", conn)

# Fetch data from the purchases table
purchases_df = pd.read_sql_query("SELECT * FROM purchases", conn)

# Fetch data from the interactions table
interactions_df = pd.read_sql_query("SELECT * FROM interactions", conn)

print("Customers Data:")
print(customers_df)

print("\nPurchases Data:")
print(purchases_df)

print("\nInteractions Data:")
print(interactions_df)



Customers Data:
     id             name                        email               phone
0     1     Jacob Harris   franklindaniel@example.com        298.963.3654
1     2   Melvin Stanley  gonzalezpatrick@example.net        601.209.0782
2     3     Dustin Clark         lauren18@example.net   487-396-8116x9316
3     4  Veronica Ibarra        jeffrey52@example.net     +1-592-271-4063
4     5       Gary Ellis    emilymitchell@example.com       (679)222-7796
..  ...              ...                          ...                 ...
95   96    Sherry Wagner           eklein@example.net        548.579.0005
96   97      Valerie Lee     jessicabrown@example.org    973-881-3042x405
97   98   Monica Collier  valenciakenneth@example.com  (537)844-0950x8050
98   99     Oscar Wilson         juanhall@example.com   (355)875-2707x321
99  100    Adam Anderson         zwebster@example.com   614.596.1088x7781

[100 rows x 4 columns]

Purchases Data:
      id  customer_id       item  amount purchase_date


In [ ]:
import pandas as pd
import sqlite3

def load_data(db_file):
    conn = sqlite3.connect(db_file)
    customers = pd.read_sql_query("SELECT * FROM customers", conn)
    purchases = pd.read_sql_query("SELECT * FROM purchases", conn)
    interactions = pd.read_sql_query("SELECT * FROM interactions", conn)
    conn.close()
    return customers, purchases, interactions

# Load data
customers, purchases, interactions = load_data("customer_data.db")

# Print column names to verify
print("Customers columns:", customers.columns)
print("Purchases columns:", purchases.columns)
print("Interactions columns:", interactions.columns)

# Ensure the column names are correct before merging
data = customers.merge(purchases, left_on='id', right_on='customer_id', suffixes=('_customer', '_purchase'))
data = data.merge(interactions, left_on='id_customer', right_on='customer_id', suffixes=('', '_interaction'))

# Print data to verify the merge
print(data.head())

# Handle missing values
data.fillna(method='ffill', inplace=True)

# Encode categorical variables
le = LabelEncoder()
data['interaction_type'] = le.fit_transform(data['interaction_type'])

# Scale numerical features
scaler = StandardScaler()
data[['amount']] = scaler.fit_transform(data[['amount']])


Customers columns: Index(['id', 'name', 'email', 'phone'], dtype='object')
Purchases columns: Index(['id', 'customer_id', 'item', 'amount', 'purchase_date'], dtype='object')
Interactions columns: Index(['id', 'customer_id', 'interaction_type', 'interaction_date', 'notes'], dtype='object')
   id_customer            name                        email         phone  \
0            1    Jacob Harris   franklindaniel@example.com  298.963.3654   
1            1    Jacob Harris   franklindaniel@example.com  298.963.3654   
2            2  Melvin Stanley  gonzalezpatrick@example.net  601.209.0782   
3            2  Melvin Stanley  gonzalezpatrick@example.net  601.209.0782   
4            2  Melvin Stanley  gonzalezpatrick@example.net  601.209.0782   

   id_purchase  customer_id       item  amount purchase_date  id  \
0            1            1      chair   15.81    2024-04-14   1   
1            1            1      chair   15.81    2024-04-14   2   
2            2            2  newspaper  150

In [ ]:
# Feature engineering
X = data[['amount', 'interaction_type']]

# Create a binary target variable based on purchase amount
threshold = 0  # Set a threshold for high-value purchases
y = (data['amount'] > threshold).astype(int)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))

Accuracy: 0.9827586206896551


In [ ]:
from transformers import pipeline

# Load a pre-trained model for sentiment analysis (as an example)
nlp = pipeline('sentiment-analysis')

# Example customer inquiry
customer_inquiry = "I am happy with the recent purchase I made."

# Analyze sentiment
result = nlp(customer_inquiry)
print(result)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998793601989746}]
